In [1]:
# libraries
from ase.io import read, write
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.io.vasp import Poscar, Outcar
import sys, os, json 

sys.path.append('../Modules')

In [3]:
from VASPDataParser import VASPDataParser

In [4]:
# get all the folders from CE_Vasp_Jobs_T3

data_path = '../Archived_Vasp_Jobs/CE_Vasp_Jobs_T3'

# get all the folders from CE_Vasp_Jobs_T3
folders = [x[0] for x in os.walk(data_path)]


In [ ]:
# need to handle the situation where i have ionic steps, create a structure, energy, force dictionary entry for each ionic step
data = {}
for folder in folders:
    parser = VASPDataParser()
    structures = parser.get_structures(os.path.join(folder, 'XDATCAR'))
    energies = parser.get_energy_without_entropy(os.path.join(folder, 'OUTCAR'))
    forces = parser.get_forces(os.path.join(folder, 'OUTCAR'))
    if 'OUTCAR-original' in os.listdir(folder):
        og_structures = parser.get_structures(os.path.join(folder, 'XDATCAR-original'))
        og_energies = parser.get_energy_without_entropy(os.path.join(folder, 'OUTCAR-original'))
        og_forces = parser.get_forces(os.path.join(folder, 'OUTCAR-original'))
        structures = og_structures + structures
        energies = og_energies + energies
        forces = og_forces + forces
    
    # strip the number from the folder name 
    folder_name = folder.split('/')[-1]
    data[str(folder_name)] = {'structures': structures, 'energies': energies, 'forces': forces}
    
    



In [6]:
# for each folder in CE_Vasp_Jobs_T3, check if the 
test_path = '../Archived_Vasp_Jobs/CE_Vasp_Jobs_T3/job_180'

# define VASPDataParser object
data_parser = VASPDataParser()

og_structures = data_parser.get_structures(os.path.join(test_path,'XDATCAR-original'))
# read the original XDATCAR and OUTCAR file
og_energies = data_parser.get_energy_without_entropy(os.path.join(test_path,'OUTCAR-original'))
og_forces = data_parser.get_forces(os.path.join(test_path,'OUTCAR-original'))

In [11]:
structures = data_parser.get_structures(os.path.join(test_path,'XDATCAR'))
energies = data_parser.get_energy_without_entropy(os.path.join(test_path,'OUTCAR'))
forces = data_parser.get_forces(os.path.join(test_path,'OUTCAR'))

In [12]:
print(len(structures))
print(len(energies))
print(len(forces))

12
12
12


In [3]:
import sys 
sys.path.append('../Modules')
from VASPDataParser import VASPDataParser

# define VASPDataParser object
data_parser = VASPDataParser()

# parse an outcar file 
outcar_path = '../Archived_Vasp_Jobs/VCrTiWZr_Summit/supercell_gen4_comp52_struct3_vac_site3_start/OUTCAR-4'

# get the stress tensor 
stress_tensor = data_parser.get_stresses(outcar_path, si_unit=False)

print(stress_tensor[-1])

[[-0.00183, 0.01887, 0.02349], [0.01887, 0.0121, -0.01515], [0.02349, -0.01515, -0.02325]]


In [10]:
import numpy as np

def calculate_kpoints(lattice_vectors, kspacing):
    """
    Calculate the equivalent N x N x N k-point grid for a given KSPACING.

    Parameters:
    lattice_vectors (np.array): 3x3 array of lattice vectors (rows) in Å.
    kspacing (float): KSPACING value in Å^-1.

    Returns:
    np.array: The number of k-points in each direction (N1, N2, N3).
    """
    # Calculate reciprocal lattice vectors
    rec_lattice_vectors = 2 * np.pi * np.linalg.inv(lattice_vectors).T

    # Calculate the magnitude of each reciprocal lattice vector
    b_lengths = np.linalg.norm(rec_lattice_vectors, axis=1)

    # Calculate the number of k-points in each direction
    kpoints = np.floor(b_lengths / kspacing + 0.5).astype(int)

    return kpoints

# Example lattice vectors (in Å)
lattice_vectors = np.array([
    [5.0, 0.0, 0.0],
    [0.0, 5.0, 0.0],
    [0.0, 0.0, 5.0]
])

lattice_vectors = np.array([
    [9.906341000, 0.000216000, -3.497101000],
    [-4.950387000, 8.548172000, -3.489774000],
    [-0.008758000, -0.006193000, 10.466536000]])

# Example KSPACING value
kspacing = 0.12

# Calculate equivalent k-points
kpoints = calculate_kpoints(lattice_vectors, kspacing)
print(f"Equivalent k-points grid: {kpoints[0]} x {kpoints[1]} x {kpoints[2]}")

Equivalent k-points grid: 6 x 6 x 6
